In [ ]:
import json
import numpy as np
import random
from tqdm.auto import tqdm
import itertools
import os
from copy import deepcopy
import matplotlib.pyplot as plt
from collections import defaultdict
import string
import shutil


In [2]:
def complete(s):
    if not s.endswith(">"):
        s = s + ">"
    if not s.startswith("<"):
        s = "<" + s
    return s

In [3]:
DATA_ROOT = "/data/locus/project_data/project_data2/chenwu2/creativity_data"

In [4]:
NUM_A = 5
NUM_B_PER_A = 500
HASH_STR_LEN = 10

In [5]:
pad_token = "<|endoftext|>"

In [ ]:
# downsampling train_inferred
for training_size in [50000]:
    print(f"training size: {training_size}")
    base_dataset_name = "sibling.{}.{}.{}.{}".format(NUM_A, NUM_B_PER_A, HASH_STR_LEN, training_size)
    dataset_name = "sibling_hybrid.{}.{}.{}.{}".format(NUM_A, NUM_B_PER_A, HASH_STR_LEN, training_size)
    # Copy the base dataset to the new dataset
    shutil.copytree(os.path.join(DATA_ROOT, base_dataset_name), os.path.join(DATA_ROOT, dataset_name))

    train_sequences = json.load(open(os.path.join(DATA_ROOT, base_dataset_name, "train.json"), "r", encoding='utf-8'))
    # Add the hybrid data to the train_sequences
    hybrid_sequences = []
    for train_sequence in train_sequences:
        input_text = train_sequence["input_text"]
        target_text = train_sequence["target_text"]
        output_text = target_text[len(input_text):]
        output_text = output_text.split("</a>")[0]
        b1, b2, a = output_text.split("><")
        b1 = complete(b1)
        b2 = complete(b2)
        a = complete(a)
        mtp_sequence_1 = {
            "input_text": input_text + pad_token,
            "target_text": input_text + pad_token + b2
        }
        hybrid_sequences.append(mtp_sequence_1)
        mtp_sequence_2 = {
            "input_text": input_text + pad_token + pad_token,
            "target_text": input_text + pad_token + pad_token + a
        }
        hybrid_sequences.append(mtp_sequence_2)
    train_sequences.extend(hybrid_sequences)

    with open(os.path.join(DATA_ROOT, dataset_name, "train.json"), "w", encoding='utf-8') as f:
        json.dump(train_sequences, f)
    